<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/data"

Mall_Customers.csv  pdf_to_excel.pdf


In [0]:
pip install tika

In [5]:
import tika
tika.initVM()
from tika import parser
parsed = parser.from_file('/content/gdrive/My Drive/data/pdf_to_excel.pdf')
print(parsed["metadata"])
print(parsed["content"])

{'Content-Type': 'application/pdf', 'Creation-Date': '2019-06-02T07:29:18Z', 'Keywords': '', 'Last-Modified': '2019-06-02T07:29:18Z', 'Last-Save-Date': '2019-06-02T07:29:18Z', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:parse_time_millis': '90', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_degraded': 'true', 'access_permission:extract_content': 'true', 'access_permission:extract_for_accessibility': 'true', 'access_permission:fill_in_form': 'true', 'access_permission:modify_annotations': 'true', 'created': '2019-06-02T07:29:18Z', 'date': '2019-06-02T07:29:18Z', 'dc:format': 'application/pdf; version=1.3', 'dc:subject': '', 'dc:title': 'Microsoft Word - Documents.docx', 'dcterms:created': '2019-06-02T07:29:18Z', 'dcterms:modified': '2019-06-02T07:29:18Z', 'meta:creation-date': '2019-06-02T07:29:18Z', 'meta:keyword': '',

In [0]:
print(parsed["content"])










































Microsoft Word - Documents.docx


File 1: 
File Name: Instructions.docx 
File Size: 2 MB 
Download Link: https://www.sendspace.com/file/r59by6 
Delete File 
Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c
1f 
 
File 2: 
File Name: excel instructions.xlsx 
File Size: 995.5 KB 
Download Link: https://www.sendspace.com/file/lv6xjf 
Delete File 
Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690
6 
 
File 3: 
File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-
OCR.pdf 
File Size: 173.3 MB 
Download Link: https://www.sendspace.com/file/mfj1z7 
Delete File 
Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27
81 
	





In [0]:
import re
content = parsed["content"]
parsed = parser.from_file('/content/gdrive/My Drive/data/pdf_to_excel.pdf')
content = parsed["content"]
content = content.replace("Microsoft Word - Documents.docx", "")
content = re.sub("File [0-9]+:", "---", content).strip()
ls = content.split("---")
d = {}

i = 1
for l in ls:
  d["File "+str(i)] = l
  i = i+1
    
for k, v in d.items():
  for line in v.splitlines():
    print(line.split(" : "))

[' ']
['File Name: Instructions.docx ']
['File Size: 2 MB ']
['Download Link: https://www.sendspace.com/file/r59by6 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/r59by6/57844a66dd1958f10f513c28bfe30c']
['1f ']
[' ']
[' ']
['File Name: excel instructions.xlsx ']
['File Size: 995.5 KB ']
['Download Link: https://www.sendspace.com/file/lv6xjf ']
['Delete File ']
['Link: https://www.sendspace.com/delete/lv6xjf/7138a5de96bf4bf22d982b53dc56690']
['6 ']
[' ']
[' ']
['File Name: L290-AQ-MDR-13850_0_SECTN_88OF107_IPX Reviewed_AC-']
['OCR.pdf ']
['File Size: 173.3 MB ']
['Download Link: https://www.sendspace.com/file/mfj1z7 ']
['Delete File ']
['Link: https://www.sendspace.com/delete/mfj1z7/1c77539be54f2415c7a24ce5427a27']
['81']


In [0]:
file = open("/content/gdrive/My Drive/data/pdf_to_excel.txt","w") 
file.write(content)
file.close()

In [0]:
match = re.match(r"(^File [0-9]+):([^File [0-9]+].*)", content, re.I)
if match:
    print("match")
    items = match.groups()

items
# re.split("File d+", content)
# key, value = content.split("File ")
# my_dict = {key: value}

# dict(zip(*content.split(':')))
# dict(zip(*iter(content.split(':'))))

('File 1',)